In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error as mse

* знайдіть найкращі параметри w⃗ для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;

In [2]:
file_data = pd.read_csv("Housing.csv")
file_data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
X = file_data[["area", "bedrooms", "bathrooms"]]
X.insert(0, "ones", 1)
X = np.array(X)
y = np.array(file_data["price"])
print(f"X.shape: {X.shape}")
print(f"y.shape: {y.shape}")

X.shape: (545, 4)
y.shape: (545,)


##### Аналітичне рішення

In [4]:
def analytic_algorithm(X: np.ndarray, y: np.ndarray):
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

analytic_variant = analytic_algorithm(X, y)
print(f"Linear regression coef.: {analytic_variant}")

Linear regression coef.: [-1.73171608e+05  3.78762754e+02  4.06820034e+05  1.38604950e+06]


#####  Sklearn рішення

In [5]:
linear_regression = LR()
linear_regression.fit(X, y)

print(f"Mean squared error: {mse(y, linear_regression.predict(X))}")
print(f"Linear regression coef.: {linear_regression.coef_}")
print(f"Linear regression intercept.: {linear_regression.intercept_}")

Mean squared error: 1791170049977.319
Linear regression coef.: [0.00000000e+00 3.78762754e+02 4.06820034e+05 1.38604950e+06]
Linear regression intercept.: -173171.6076326333


##### * напишіть функцію гіпотези лінійної регресії у векторному вигляді;
##### * створіть функцію для обчислення функції втрат у векторному вигляді;
##### * реалізуйте один крок градієнтного спуску;

In [6]:
def cost(y: np.array, h: np.array) -> float:
    return np.mean((h - y) ** 2) / 2


class LinearRegression:
    def __init__(self, lr: float = 0.001, thr: float = 0.00001, n_epochs: int = 100, w: list = None):
        self.lr = lr
        self.thr = thr
        self.n_epochs = n_epochs
        self.w = np.array(w) if w else np.array([])

    def predict(self, X: np.ndarray) -> np.ndarray:
        return X @ self.w

    def update_w(self, X: np.ndarray, y: np.ndarray):  # крок градієнтного спуску
        h = self.predict(X)
        self.w -= self.lr / len(y) * X.T @ (h - y)

    def fit(self, X: np.ndarray, y: np.ndarray):
        if len(self.w) != len(X[0]):
            self.w = np.array([np.random.uniform(-5, 5)] * len(X[0]))

        last_cost = 1e+40
        count = 0
        for i in range(self.n_epochs):
            count += 1
            self.update_w(X, y)
            new_cost = cost(y, self.predict(X))

            cost_delta = last_cost - new_cost
            if cost_delta < self.thr:
                break
            last_cost = new_cost
        print(f"{count}, {self.w}, {last_cost}")

##### Лінійна регресія без маштабування та нормалізації данних

In [7]:
init_w = [-1.73e+05, 3.78e+02, 4.06e+05, 1.38e+06]
linear_regression = LinearRegression(lr=0.00000001, n_epochs=400000, thr=0.001, w=init_w)
linear_regression.fit(X, y)

400000, [-1.72994947e+05  3.80453504e+02  4.06018444e+05  1.38001161e+06], 895595491066.5187


##### Лінійна регресія з маштабуванням та нормалізацією данних

In [8]:
y_mean = y.mean()
y_range = y.max() - y.min()
y_scales = (y - y_mean) / y_range
pd.DataFrame(y_scales).head()

,0
0,0.738811
1,0.647902
2,0.647902
3,0.644872
4,0.575175


In [9]:
pd.DataFrame(X).head()

,0,1,2,3
0,1,7420,4,2
1,1,8960,4,4
2,1,9960,3,2
3,1,7500,4,2
4,1,7420,4,1


In [10]:
x_means = X[:, 1:].mean(axis=0)
x_means = np.insert(x_means, 0, 0)
print(f"x_means: {x_means}")

x_ranges = X[:, 1:].max(axis=0) - X[:, 1:].min(axis=0)
x_ranges = np.insert(x_ranges, 0, 1)
print(f"x_ranges: {x_ranges}")

X_scales = (X - x_means) / x_ranges
pd.DataFrame(X_scales).head()

x_means: [0.00000000e+00 5.15054128e+03 2.96513761e+00 1.28623853e+00]
x_ranges: [    1 14550     5     3]


,0,1,2,3
0,1.0,0.155977,0.206972,0.237920
1,1.0,0.261818,0.206972,0.904587
2,1.0,0.330547,0.006972,0.237920
3,1.0,0.161475,0.206972,0.237920
4,1.0,0.155977,0.206972,-0.095413


In [11]:
init_w = [-1.73e+05, 3.78e+02, 4.06e+05, 1.38e+06]
linear_regression = LinearRegression(lr=0.00000001, n_epochs=400000, thr=0.001, w=init_w)
linear_regression.fit(X_scales, y_scales)

400000, [-1.72309382e+05  3.45872450e+02  4.05913743e+05  1.37983045e+06], 48464679554.692696


### Порівняння результатів

In [12]:
# Аналітичне рішення
[-1.73171608e+05  3.78762754e+02  4.06820034e+05  1.38604950e+06]
# Sklearn рішення
[-1.73171608e+05  3.78762754e+02  4.06820034e+05  1.38604950e+06]
# Лінійна регресія без маштабування та нормалізації данних 400000 циклів 
[-1.72994947e+05  3.80453504e+02  4.06018444e+05  1.38001161e+06], 895595491066.5187
# Лінійна регресія з маштабуванням та нормалізацією данних 400000 циклів 
[-1.72309382e+05  3.45872450e+02  4.05913743e+05  1.37983045e+06]


### Висновок

Рузультати аналітичного рішення та sklearn рішення повністю співпадають по значенням коєфіцієнтів та приблизно мають однакову швидкість виконання (заміряти швидкість не вистачило часу). Таке враження, що sclearn використав для розрахунку аналітичне рішення). Був неприємно вражений різницею результатів прикладу з лекції та прикладу з ДЗ при розрахунку за допомогою лінійної регресії. З ростом набору данних та особливо кількості ознак різко зростає кількість ітерацій для знаходження мінімуму функції втрат. Навіть використання масштабування не приносить особливого виграшу окрім швидкості розрахунку. Можливо на більших обчислювальних потужностях все не так сумно. Але бібліотека Sklearn дає шанс моєму ноутбуку, якщо не використовує аналітичне рішення). Забув вказати, що не зовсім є зрозумілим вибір початкових коефіцієнтів, якщо не орієнтуватися на аналітичне рішення та  Sklearn.